# Statistical Evaluation of ABX Test with webMUSHRA


Below we read the csv-data of the [webMUSHRA](https://github.com/audiolabs/webMUSHRA) based ABX-test and analyze the data in terms of binomial and $\chi^2$ statistics

We used [GPower3](http://www.gpower.hhu.de/) to configure the ABX test design, see
Faul, F., Erdfelder, E., Lang, A.-G., & Buchner, A. (2007). *GPower 3: A flexible statistical power analysis program for the social, behavioral, and biomedical sciences*. Behavior Research Methods, **39**, 175-191

VP...Versuchsperson, i.e. test subject

In [ ]:
import csv
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import binom, chi2, chisquare, chi2_contingency

def calc_p(correct_trials, total_trials):
    # return probability of at least 'correct_trials' out of 'total_trials'
    # with 50:50 chance coin flipping
    return 1-binom.cdf(correct_trials-1, total_trials, 0.5)

def print_quartiles(tmp, unit=' '):  # some basic statistics 
    # tmp = 1 * np.random.randn(1, 100000) + 0
    print('mean: %3.1f'% np.mean(tmp), unit)
    print('std : %3.1f'% np.std(tmp, ddof=1), unit)
    print('percentiles:')
    print('P5  : %3.1f'% np.quantile(tmp, .05), unit)
    print('P25 : %3.1f'% np.quantile(tmp, .25), unit)
    print('P50 : %3.1f'% np.quantile(tmp, .5), unit)
    print('P75 : %3.1f'% np.quantile(tmp, .75), unit)
    print('P95 : %3.1f'% np.quantile(tmp, .95), unit)
    print('IQR : %3.1f'% (np.quantile(tmp, .75) - np.quantile(tmp, .25)), unit)
    print()

## Test Characteristics for 5 Audio Files Repeated Measures

In [ ]:
print(0.05/5, 1-0.05/5)
print(19/25)
print(25*5)

## Test Characteristics for Single Audio File, Very Small Effect Size

Single audio with about same number of total trials 119 (vs. 25*5=125 from five stimuli), leading to decreased (for certain stimuli probably more realistic) effect size:

## Test Characteristics for Single Audio File, 18/26 Trials

If only a single audio was to be evaluated by a single VP with 25 trials as above, we consider **26** trials yielding decreased effect size of 0.31 for an alpha = 0.05 and 1-beta about 0.95.

Or: g=0.3 results in 1-b = 0.89 for 18/25 trials

In [ ]:
print(18/26)

## Prep Data Base

In [ ]:
# note: in csv the trial_id suffix must be between
# 0...9 for test part and 00..99 for trial number 
suffix_length = 5  # if audio is 'hotel', trial_id is denoted with
# e.g. 'hotel_2_01' for part 2, stimulus 01 
csvfile = '../results/abx_constant_phase_shift/paired_comparison.csv'
audio = ['square_a',
         'square_b',
         'pnoise',
         'castanets',
         'hotelcalifornia']

## Make Sorted Dictionaries

To make things more accessible we sort and write help csv-files first.

We also count the number of VP here in the first sorting.

In [ ]:
VP = 0  # init as cleared
with open(csvfile, newline='') as f:
    dr = csv.DictReader(f, delimiter=",")
    sorted_vp_id = sorted(dr, key=lambda row:(row['vp_id'], row['trial_id'], row['choice_answer']),
                          reverse=False)
with open('../results/abx_constant_phase_shift/sorted_vp_id.csv', 'w+') as f:
    fieldnames = ['vp_id', 'trial_id', 'choice_answer',
                  'age', 'gender', 'choice_reference', 'choice_non_reference',
                  'session_test_id', 'choice_time', 'choice_comment']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for row in sorted_vp_id:
        writer.writerow(row)
        if int(row['vp_id']) > VP:
            VP = int(row['vp_id'])  # 'count' VP
        
with open(csvfile,newline='') as f:
    dr = csv.DictReader(f, delimiter=",")
    sorted_trial_id = sorted(dr, key=lambda row:(row['trial_id'], row['choice_answer'], row['vp_id']),
                             reverse=False)
with open('../results/abx_constant_phase_shift/sorted_trial_id.csv', 'w+') as f:
    fieldnames = ['trial_id', 'choice_answer', 'vp_id',
                  'age', 'gender', 'choice_reference', 'choice_non_reference',
                  'session_test_id', 'choice_time', 'choice_comment']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for row in sorted_trial_id:
        writer.writerow(row)

VP = VP+1 # number of in listening test persons, in csv encoded as vp_id = 0...VP-1
print('total VPs =',VP)

## Statistics: Age in Years

In [ ]:
age = np.zeros((VP, 1), dtype=np.uint)
for vp in range(0, VP):
    for row in sorted_vp_id:
        if int(row['vp_id']) == vp:
            age[vp] = int(row['age'])
            break
print_quartiles(age, unit='y')

## Statistics: Trial Decision in Seconds for all VP

In [ ]:
t = np.zeros((VP*25,5))
for ai, a in enumerate(audio):  # for all audio
    print('audio:', a)
    tmp = np.empty(0)
    for row in sorted_trial_id:
        if row['trial_id'][0:-suffix_length] == a:
            tmp = np.append(tmp, int(row['choice_time']))
    print('total time %3.1f' % (np.sum(tmp)/1000/60), 'min')
    # note: the recorded time might include longer rests that VP took while rating!
    # however, we might assume that these breaks were equally distributed over the
    # trials, so the median and IQR should give a fair picture of rating times
    print_quartiles(tmp/1000, unit='s')
    t[:,ai] = tmp

In [ ]:
medianprops = dict(linestyle='-', linewidth=3, color='C0')
boxprops = dict(linestyle='-', linewidth=1.5, color='k')
flierprops = dict(marker='P', markerfacecolor='C0',
                  markeredgecolor='C0',
                  markersize=4,
                  linestyle='none')
plt.boxplot(t/1000, whis=[5,95], labels=audio, widths=0.75,
            medianprops=medianprops,
            boxprops=boxprops,
            flierprops=flierprops);
plt.ylim(0,150)
plt.yticks(np.arange(0,160,10))
plt.ylabel('trial decision time in seconds')
plt.title(r'median P$_{50}$, box P$_{25}$ & P$_{75}$, whisker P$_{5}$ & P$_{95}$')
plt.grid(True);
plt.savefig('boxplot_time.png')

## Data Matrix [VP x Audio]

* total trials
* correct trials
* incorrect trials

In [ ]:
x_total = np.zeros((VP, len(audio)), dtype=np.uint)
x_correct = np.zeros((VP, len(audio)), dtype=np.uint)
x_incorrect = np.zeros((VP, len(audio)), dtype=np.uint)
for vp in range(0, VP):  # for all participants
    for ai, a in enumerate(audio):  # for all audio
        for row in sorted_vp_id:  # through data
            if (int(row['vp_id']) == vp) and (row['trial_id'][0:-suffix_length] == a):
                # data match:
                x_total[vp, ai]+= 1
                if row['choice_answer'] == 'correct':
                    x_correct[vp, ai]+= 1
                elif row['choice_answer'] == 'incorrect':
                    x_incorrect[vp, ai]+= 1
if False:  # test data
    x_correct[0,0] = 10  # p = 0.019287109375
    x_incorrect[0,0] = 2
    x_total[0,0] = 12
    x_correct[7,3] = 7  # p = 0.03515625
    x_incorrect[7,3] = 1
    x_total[7,3] = 8
    x_correct[2,2] = 98  # p = 0.04705736150777151
    x_incorrect[2,2] = 173-98
    x_total[2,2] = 173

if (x_total==0).any() and True:
    print('!!! warning !!! check: x_total is zero somewhere')
    print('x_total==0 should not happen (i.e. all False is required):\n', x_total==0)
    
if np.max(np.abs(x_total - x_correct - x_incorrect)) != 0:
    print('!!! warning !!! check: x_total != x_correct + x_incorrect results')

if VP*5*25!=np.sum(x_total):
    print('!!! warning !!! missing data, check that all VP performed all parts')

print(x_total)
    
print()

if False:
    print('audio:\n', audio)
    print('x_total:\n', x_total)
    print('x_correct:\n', x_correct)
    print('x_incorrect:\n', x_incorrect)
    print()

# per audio:
idx = 0
print('all vp per', audio[idx], ': total', np.sum(x_total[:,idx]),
      ', correct', np.sum(x_correct[:,idx]),
      ', incorrect', np.sum(x_incorrect[:,idx]), ', p = %4.3f' %
      calc_p(np.sum(x_correct[:,idx]), np.sum(x_total[:,idx])))

# per vp:
idx = 0
print('all audio per vp', idx, ': total', np.sum(x_total[idx,:]),
      ', correct', np.sum(x_correct[idx,:]),
      ', incorrect', np.sum(x_incorrect[idx,:]), ', p = %4.3f' %
      calc_p(np.sum(x_correct[idx,:]), np.sum(x_total[idx,:])))

## Check Individual Probabilities

In [ ]:
p = calc_p(x_correct, x_total)
pb = p<0.01
h = x_correct / x_total * 100  # detection frequency correct answers / Häufigkeit 
np.set_printoptions(precision=1, suppress=True)
print('audio:\n', audio)
print('detection rate in % :\n', h)
#print('pbinom = \n', p)
print('pbinom < 1% = (i.e. we may reject H0, i.e. guessing is very unlikely, ')
print('i.e. it is very unlikely that our data stems from the H0 PDF(50:50 chance)):\n', pb)
np.set_printoptions(precision=8, suppress=False)

## Scatter Plot

In [ ]:
#TBD: dynamic alloc wrt len(audio)
data_dict = {0: list(h[:,0]), 1: list(h[:,1]), 2: list(h[:,2]), 3: list(h[:,3]), 4: list(h[:,4])}

alabel = audio.copy()
for (c, v) in enumerate(alabel):
    if v == 'square_a':
        alabel[c] = 'square -90°'
    if v == 'square_b':
        alabel[c] = 'square -45°'  
    if v == 'pnoise':
        alabel[c] = 'pink noise -90°'  
    if v == 'hotelcalifornia':
        alabel[c] = 'hotel cal -90°'
    if v == 'castanets':
        alabel[c] = 'castanets -90°'        

fig = plt.figure()    
ax = fig.add_subplot(1,1,1)
# 19/25 hardcoded:
plt.plot([-1, len(audio)],[19/25*100, 19/25*100], 
         '-.', color='k', lw=2, label=r'$H_0$ rejection border, 19 correct of 25 trials, i.e. 76%', zorder=1)
plt.plot([-1, len(audio)], [50, 50], '-', color='k', lw=1, label=r'$H_0(p=0.5)$, i.e. guessing 50%', zorder=1)
size_constant = 30
for xe, ye in data_dict.items():
    xAxis = [xe] * len(ye)
    sizes = [ye.count(num)**1.66 * size_constant for num in ye]
    counter = [ye.count(num) for num in ye]
    plt.scatter(xAxis, ye, s=sizes, color='C0',zorder=3)
    for c, v in enumerate(counter):  # annotate totals>1 as text
        if v > 1:
            plt.text(xe-0.03, ye[c]-1, v,color='w')
plt.xticks(np.arange(0,len(audio)), [alabel[0], alabel[1], alabel[2], alabel[3], alabel[4]])
ty = np.round(np.arange(5,26)/25*100, 1)  # y ticks, 25 hardcoded!
tys = ty.astype(str)  # y ticks label as strings
tys[1::2] = ' '  # only each second string
plt.yticks(ty, tys)
plt.xlim(-0.15,len(audio)-1+0.15)
plt.grid(True)
ax.set_axisbelow(True)
plt.xlabel('stimulus')
plt.ylabel('correct answers in %')
plt.title(r'effect size g=0.4, $\alpha=0.01$, power $1-\beta=0.99$, $N=25$, $N_\mathrm{crit}=19$')
plt.legend(loc=3);

ax2 = ax.twinx()
ty = np.arange(0,22)  # y ticks, hack!!!
tys = (ty+5).astype(str)  # y ticks label as strings
tys[1::2] = ' '  # only each second string
plt.yticks(ty, tys)
plt.ylabel('number of correct answers');

plt.savefig('scatter.png')

## Chi2 Tests Single Items

In [ ]:
if False:
    # manual calc for audio[0]:
    idx = 0
    total = sum(x_total)[idx]  # total answers
    expect = total*0.5 # 50:50
    incorrect = sum(x_incorrect)[idx]  # incorrect answers
    correct = sum(x_correct)[idx]  # correct asnwers
    chisq = ((correct-expect)**2 / expect) + ((incorrect-expect)**2 / expect)
    p = 1 - chi2.cdf(chisq, df=1)
    print('audio:', audio[idx])
    print('incorrect %4.3f' % (incorrect/total), ', correct %4.3f' % (correct/total))
    print(chisquare([correct, incorrect]))
    print('chi2 = %4.3f'% chisq, ', p = %4.3f'% p, ', p<0.01', p<0.05)
    print()

#doing all audio with chisquare():
for (c,v) in enumerate(audio):
    print('audio:', v)
    print('correct answers:', sum(x_correct)[c], 'incorrect answers', sum(x_incorrect)[c])
    chisq, p = chisquare([sum(x_correct)[c], sum(x_incorrect)[c]])
    print('chi2 = %4.3f'% chisq, ', p = %4.3f'% p, ', p<0.01', p<0.01)
    if p<0.01:
        print('reject H0, frequency of correct and incorrect answers is different')
        print('indicates NO guessing')
    else:
        print('H0 might not be rejected, frequency of correct and incorrect answers is about the same')
        print('indicates guessing')
    print()

## Chi2 Tests Pairwise Comparison

In [ ]:
# helpful stuff: http://www.biostathandbook.com/chiind.html
if False:
    f_obs = np.array([[8177, 8147], [575, 555]]).T  # Selenium vs. Selenium+E
    chisq, p, df, ex = chi2_contingency(f_obs, correction=False)
    print('chi2 = %4.3f' % chisq, ', p = %4.3f'% p, ', p<0.05', p<0.05)
    chi2_contingency(f_obs, correction=False)

In [ ]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html
cnt = 0
n_of_pairs = 10
for i in range(0,len(audio)):  # find all unique pairs
    for j in range(i,len(audio)): 
        if i!=j:
            print('pair', cnt, ':', audio[i], 'vs.', audio[j])  # unique pair
            cnt += 1
            obs = [[sum(x_correct)[i], sum(x_correct)[j]], [sum(x_incorrect)[i], sum(x_incorrect)[j]]]
            chisq, p, df, ex = chi2_contingency(obs, correction=False)
            print('df =', df, ', chi2 = %4.3f'% chisq, ', p = %4.3f'% p, ', reject H0:', p<0.05/n_of_pairs)

            # print(obs)
            # print('corr1 =', obs[0][0], 'corr2 =', obs[1][0], 'incorr1 =', obs[0][1], 'incorr2 =', obs[1][1])
            corr1 = obs[0][0]
            corr2 = obs[1][0]
            incorr1 = obs[0][1]
            incorr2 = obs[1][1]
            total1 = corr1 + incorr1
            total2 = corr2 + incorr2
            OR = (corr1/incorr1) / (corr2/incorr2)
            RR = (corr1/total1) / (corr2/total2)
            print('odds ratio (OR):',  OR, ', risk ratio (RR):', RR)

            # alpha correction for 10 pairwise comparisons
            if p<0.05/n_of_pairs:
                print('reject H0, variables are rather independent, indicates different rating')
            else:
                print('H0 might not be rejected, indicates similar rating')
            print()

we might not need the stuff below anymore...

## all vp per audio

In [ ]:
print('all vp per audio:\n')
for a in audio:
    print('audio:', a)
    total_trials, correct_trials, incorrect_trials = 0, 0, 0
    for row in sorted_trial_id:
        if row['trial_id'][0:-suffix_length] == a:
            total_trials+= 1
            if row['choice_answer'] == 'correct':
                correct_trials+= 1   
            elif row['choice_answer'] == 'incorrect':
                incorrect_trials+= 1
    if total_trials - correct_trials - incorrect_trials == 0: 
        print('trials: total', total_trials,
              ', correct', correct_trials,
              ', incorrect', incorrect_trials)
        p = calc_p(correct_trials, total_trials)
        print('p = %4.3f' % p)
        if p<0.05:
            print('H0 may be rejected, i.e. guessing is very unlikely')
    else:
        print('total trials not equal to correct+incorrect trials')
    print()

## all audio per vp

In [ ]:
print('all audio per vp:\n')
for vp in range(0, VP):
    print('VP:', vp)
    total_trials, correct_trials, incorrect_trials = 0, 0, 0
    for row in sorted_vp_id:
        if int(row['vp_id']) == vp:
            total_trials+= 1
            if row['choice_answer'] == 'correct':
                correct_trials+= 1   
            elif row['choice_answer'] == 'incorrect':
                incorrect_trials+= 1
    if total_trials - correct_trials - incorrect_trials == 0: 
        print('trials: total', total_trials,
              ', correct', correct_trials,
              ', incorrect', incorrect_trials)
        p = calc_p(correct_trials, total_trials)
        print('p = %4.3f' % p)
        if p<0.05:
            print('H0 may be rejected, i.e. guessing is very unlikely')
    else:
        print('check: x_total != x_correct + x_incorrect results')
    print()

In [ ]:
if False:
    suffix_length = 3
    print('all vp per audio:\n')
    for part in range(0,4):
        print('### part', part)
        for a in audio:
            print('audio:', a)
            total_trials, correct_trials, incorrect_trials = 0, 0, 0
            for row in sorted_trial_id:
                if row['trial_id'][0:-suffix_length] == a+'_'+str(part):
                    total_trials+= 1
                    if row['choice_answer'] == 'correct':
                        correct_trials+= 1   
                    elif row['choice_answer'] == 'incorrect':
                        incorrect_trials+= 1
            if total_trials - correct_trials - incorrect_trials == 0: 
                print('trials: total', total_trials,
                      ', correct', correct_trials,
                      ', incorrect', incorrect_trials)
                p = calc_p(correct_trials, total_trials)
                print('p =',p)
                #if p<0.05:
                #    print('H0 may be rejected, i.e. guessing is very unlikely')
            else:
                tmp = 0
                #print('total trials not equal to correct+incorrect trials')
            print()